In [10]:
%run my_functions.py


C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\spacy\util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.4.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package stopwords to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downlo

In [11]:
# loading preprocessed dataset
with open("../dataframes/raw_data_Books_00_clean_and_cut.pkl", "rb") as f:
    df = pickle.load(f)
df = df[:20000]

In [12]:
texts =[]
for doc in df.cut_and_clean:
        words = tokenizer.tokenize(doc)
        texts.append(words)

dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=20, no_above=0.8)

## 1. Transformers (Bag-of-Words)

### 1.1 Count Vectorizer

In [13]:
# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.8, min_df=20, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf_fit = vectorizer.fit_transform(df['cut_and_clean'])
#pickle.dump(tf_fit, open('../models/bow_tf.pkl', 'wb')) #save fitted model
tf = tf_fit.toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

# Non-Negative Matrix Factorization

In [83]:
from sklearn.decomposition import NMF
nmf_cv = NMF(n_components=8, init= 'nndsvd')
nmf_npmi = NMF(n_components=15, init= 'nndsvd')
nmf_uci = NMF(n_components=10, init= 'nndsvd')

### NMF TF

In [84]:
NMF_tf_cv = nmf_cv.fit(tf)
NMF_tf_npmi = nmf_npmi.fit(tf)
NMF_tf_uci = nmf_uci.fit(tf)
display_topics(NMF_tf_cv, tf_feature_names, 10).to_excel("../models/nmf_tf8.xlsx")
display_topics(NMF_tf_npmi, tf_feature_names, 10).to_excel("../models/nmf_tf15.xlsx")
display_topics(NMF_tf_uci, tf_feature_names, 10).to_excel("../models/nmf_tf10.xlsx")

In [107]:
# Evaluation Metrics for whole dataset

cv_nmf_tf, c_umass_nmf_tf, c_npmi_nmf_tf, c_uci_nmf_tf, sil_score_nmf_tf, ch_index_nmf_tf, db_index_nmf_tf = evaluation(NMF_tf_cv, tf_fit, tf, tf_feature_names, 20, df["cut_and_clean"], dictionary)
cv_nmf_tf, c_umass_nmf_tf, c_npmi_nmf_tf, c_uci_nmf_tf, sil_score_nmf_tf, ch_index_nmf_tf, db_index_nmf_tf = evaluation(NMF_tf_npmi, tf_fit, tf, tf_feature_names, 20, df["cut_and_clean"], dictionary)
cv_nmf_tf, c_umass_nmf_tf, c_npmi_nmf_tf, c_uci_nmf_tf, sil_score_nmf_tf, ch_index_nmf_tf, db_index_nmf_tf = evaluation(NMF_tf_uci, tf_fit, tf, tf_feature_names, 20, df["cut_and_clean"], dictionary)



Coherence Score CV:  0.5813181284414983

Coherence Score UMass:  -2.820633396457637

Coherence Score NPMI:  0.03183704242631619

Coherence Score UCI:  -0.40862802087196237

Silhouette Score:  -0.08901912641548664

Calinski-Harabasz Score:  32.81653270919548

Davies-Bouldin Score:  7.541319938842041


In [108]:
data = display_topics(NMF_tf_npmi, tf_feature_names, 10)
fig = make_subplots(rows=5, cols=3, subplot_titles= ["Travel / Language", "Education", "Book Cover", "Religion / Belief / Healing", "Japanese Society (?)", "Family", "Politics", "Book Plot", "Cooking / Recipes", "Fanatasy Novel (?)","Workout Exercises","Unknown", "Science / Life", "Art / Music", "Christianity"])
colors = ["Blue", "Pink", "SeaGreen", "Orange", "Grey", "Purple", "Red", "LightBlue", "LightBlue", "LightBlue", "LightBlue", "LightBlue", "LightBlue", "LightBlue", "LightBlue"]
for col in range(0,  int(len(data.columns)) ,2):
    if col < 10:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"), col= 1, row = int(col/2)+1)
    if col >= 10 and col < 20:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"),col= 2, row = int(col/2)-4)
    if col >= 20:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col], 
        marker_color=colors[0], orientation = "h"), col= 3, row = int(col/2)-9)

fig.update_layout( height=1800, width=1600, title_text="Final NMF TF - Topic Keywords ranked by Weight", showlegend=False, titlefont = dict(size = 20),
    )
fig.update_annotations(font_size=20)
fig.update_yaxes( type = "category" , autorange = "reversed", tickfont = dict(size=16))
fig.update_xaxes( title = 'Weight', range = (0,10)  ,tickfont = dict(size=16))